<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Row" data-toc-modified-id="Row-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Row</a></span></li><li><span><a href="#Colonnes" data-toc-modified-id="Colonnes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Colonnes</a></span></li><li><span><a href="#expr" data-toc-modified-id="expr-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>expr</a></span></li><li><span><a href="#Transformations" data-toc-modified-id="Transformations-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Transformations</a></span><ul class="toc-item"><li><span><a href="#Projections" data-toc-modified-id="Projections-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Projections</a></span><ul class="toc-item"><li><span><a href="#selectExpr" data-toc-modified-id="selectExpr-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>selectExpr</a></span></li><li><span><a href="#drop" data-toc-modified-id="drop-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>drop</a></span></li><li><span><a href="#orderBy" data-toc-modified-id="orderBy-4.1.3"><span class="toc-item-num">4.1.3&nbsp;&nbsp;</span>orderBy</a></span></li></ul></li></ul></li></ul></div>

In [25]:
import os
from pyspark.sql.functions import *
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

In [26]:
spark

# Row
Un DataFrame est un objet de type **« Dataset[Row] »**, ainsi toutes les caractéristiques de l’objet **« Dataset »** sont également valables pour les **DataFrames**.

In [27]:
from pyspark.sql import Row
from pyspark.sql.functions import *

uneLigne = Row(1,"Isabelle","BIZOÏ")
uneAutre = Row(2,"Razvan","BIZOÏ")
listeLignes = [uneLigne,uneAutre]
personnes = spark.createDataFrame(listeLignes,["Id","Prenom","Nom"])
personnes.show()

+---+--------+-----+
| Id|  Prenom|  Nom|
+---+--------+-----+
|  1|Isabelle|BIZOÏ|
|  2|  Razvan|BIZOÏ|
+---+--------+-----+



# Colonnes

<img src="https://raw.githubusercontent.com/rbizoi/AnalyserLesDonneesAvecSpark/main/DataFrameSpark/images/M06-01.png" width="400">    

In [28]:
from pyspark.sql.functions import expr,col

meteoDataFrame  = spark.read.format('csv')\
    .option('sep',';')\
    .option('header','true')\
    .option('nullValue','mq')\
    .option('inferSchema', 'true')\
    .load('../donnees/meteo1')\
    .cache()

In [29]:
meteoDataFrame.printSchema()

root
 |-- numer_sta: integer (nullable = true)
 |-- date: long (nullable = true)
 |-- pmer: integer (nullable = true)
 |-- tend: integer (nullable = true)
 |-- cod_tend: integer (nullable = true)
 |-- dd: integer (nullable = true)
 |-- ff: double (nullable = true)
 |-- t: double (nullable = true)
 |-- td: double (nullable = true)
 |-- u: integer (nullable = true)
 |-- vv: integer (nullable = true)
 |-- ww: integer (nullable = true)
 |-- w1: integer (nullable = true)
 |-- w2: integer (nullable = true)
 |-- n: integer (nullable = true)
 |-- nbas: integer (nullable = true)
 |-- hbas: integer (nullable = true)
 |-- cl: integer (nullable = true)
 |-- cm: integer (nullable = true)
 |-- ch: integer (nullable = true)
 |-- pres: integer (nullable = true)
 |-- niv_bar: string (nullable = true)
 |-- geop: integer (nullable = true)
 |-- tend24: integer (nullable = true)
 |-- tn12: double (nullable = true)
 |-- tn24: double (nullable = true)
 |-- tx12: double (nullable = true)
 |-- tx24: double (nu

In [30]:
len(meteoDataFrame.columns), meteoDataFrame.columns

(60,
 ['numer_sta',
  'date',
  'pmer',
  'tend',
  'cod_tend',
  'dd',
  'ff',
  't',
  'td',
  'u',
  'vv',
  'ww',
  'w1',
  'w2',
  'n',
  'nbas',
  'hbas',
  'cl',
  'cm',
  'ch',
  'pres',
  'niv_bar',
  'geop',
  'tend24',
  'tn12',
  'tn24',
  'tx12',
  'tx24',
  'tminsol',
  'sw',
  'tw',
  'raf10',
  'rafper',
  'per',
  'etat_sol',
  'ht_neige',
  'ssfrai',
  'perssfrai',
  'rr1',
  'rr3',
  'rr6',
  'rr12',
  'rr24',
  'phenspe1',
  'phenspe2',
  'phenspe3',
  'phenspe4',
  'nnuage1',
  'ctype1',
  'hnuage1',
  'nnuage2',
  'ctype2',
  'hnuage2',
  'nnuage3',
  'ctype3',
  'hnuage3',
  'nnuage4',
  'ctype4',
  'hnuage4',
  '_c59'])

In [31]:
meteoDataFrame.select('numer_sta',"date","t").show(3)

+---------+--------------+------+
|numer_sta|          date|     t|
+---------+--------------+------+
|     7005|20200301000000|278.05|
|     7015|20200301000000|278.25|
|     7020|20200301000000|280.85|
+---------+--------------+------+
only showing top 3 rows



In [32]:
meteoDataFrame.numer_sta,meteoDataFrame["numer_sta"],meteoDataFrame.t

(Column<'numer_sta'>, Column<'numer_sta'>, Column<'t'>)

# expr
La fonction permet d’écrire directement une expression qui est exécutée pour l’ensemble des lignes.

In [33]:
meteoDataFrame.select('numer_sta',
                      expr('t  - 273.15').alias('temperature'),
                      expr('(t + pres/1000)*vv/1000').alias('calc')
        ).show(3)

+---------+------------------+-------+
|numer_sta|       temperature|   calc|
+---------+------------------+-------+
|     7005| 4.900000000000034| 7532.6|
|     7015| 5.100000000000023|22626.6|
|     7020|7.7000000000000455| 3800.4|
+---------+------------------+-------+
only showing top 3 rows



# Transformations

<img src="https://raw.githubusercontent.com/rbizoi/AnalyserLesDonneesAvecSpark/main/DataFrameSpark/images/M06-02.png" width="400"> 

## Projections

In [34]:
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

schema = StructType([
        StructField('Id'           , StringType() , True),
        StructField('ville'        , StringType() , True),
        StructField('latitude'     , FloatType() , True),
        StructField('longitude'    , FloatType() , True),
        StructField('altitude'     , IntegerType() , True)])

villes  = spark.read.format('csv')   \
      .option('sep',';')                \
      .option('mergeSchema', 'true')    \
      .option('header','true')          \
      .schema(schema)                   \
      .load('../donnees/meteo/postesSynop.csv')  \
      .cache()

@udf("string")
def formatVille(ville):
    if ville in ['CLERMONT-FD','MONT-DE-MARSAN',
                                   'ST-PIERRE','ST-BARTHELEMY METEO'] :
        return ville.title()
    else :
        if ville.find('-') != -1 :
            return ville[0:ville.find('-')].title()
        else:
            return ville.title()

villesT  = villes.select(
                col('Id').alias('id'),
                formatVille('ville').alias('ville'),
               'latitude',
               'longitude',
               'altitude')

### selectExpr

In [35]:
villes.selectExpr('*','altitude * 1000 as alt').show(3)

+-----+---------------+---------+---------+--------+-----+
|   Id|          ville| latitude|longitude|altitude|  alt|
+-----+---------------+---------+---------+--------+-----+
|07005|      ABBEVILLE|   50.136|    1.834|      69|69000|
|07015|  LILLE-LESQUIN|    50.57|   3.0975|      47|47000|
|07020|PTE DE LA HAGUE|49.725166|-1.939833|       6| 6000|
+-----+---------------+---------+---------+--------+-----+
only showing top 3 rows



In [36]:
villesT.selectExpr('*','altitude * 1000 as alt').show(3)

+-----+---------------+---------+---------+--------+-----+
|   id|          ville| latitude|longitude|altitude|  alt|
+-----+---------------+---------+---------+--------+-----+
|07005|      Abbeville|   50.136|    1.834|      69|69000|
|07015|          Lille|    50.57|   3.0975|      47|47000|
|07020|Pte De La Hague|49.725166|-1.939833|       6| 6000|
+-----+---------------+---------+---------+--------+-----+
only showing top 3 rows



In [37]:
villes.drop('Id', 'latitude', 'longitude').show(3)

+---------------+--------+
|          ville|altitude|
+---------------+--------+
|      ABBEVILLE|      69|
|  LILLE-LESQUIN|      47|
|PTE DE LA HAGUE|       6|
+---------------+--------+
only showing top 3 rows



### drop

In [38]:
villes.drop('Id', 'latitude', 'longitude')\
       .orderBy('ville').show(5)

+-------------+--------+
|        ville|altitude|
+-------------+--------+
|    ABBEVILLE|      69|
|      AJACCIO|       5|
|      ALENCON|     143|
|BALE-MULHOUSE|     263|
|       BASTIA|      10|
+-------------+--------+
only showing top 5 rows



In [39]:
villes.drop('Id', 'latitude', 'longitude')\
       .orderBy(desc('altitude')).show(5)

+------------------+--------+
|             ville|altitude|
+------------------+--------+
|            EMBRUN|     871|
|     LE PUY-LOUDES|     833|
|            MILLAU|     712|
|         ST GIRONS|     414|
|LIMOGES-BELLEGARDE|     402|
+------------------+--------+
only showing top 5 rows



In [40]:
meteo = meteoDataFrame.select(
                 col('numer_sta'),
                 to_timestamp(col('date').cast('string'),'yyyyMMddHHmmss'),
                 col('date')[0:4].cast('int') ,
                 col('date')[5:2].cast('int'),
                 col('date')[7:2].cast('int'),
                 col('date')[5:4],
                 round(col('t') - 273.15,2),
                 col('u') / 100 ,
                 col('vv') / 1000 ,
                 col('pres') / 1000,
                 coalesce( col('rr3'),
                           col('rr24')/8,
                           col('rr12')/4,
                           col('rr6')/2,
                           col('rr1')*3  ) )\
             .toDF('id','date','annee','mois','jour','mois_jour','temperature',
                   'humidite','visibilite','pression','precipitations')\
             .cache()

### orderBy

In [41]:
meteo.where('id < 8000')\
      .select ('id','annee','mois','jour','temperature')\
      .orderBy( 'id','annee','mois','jour','temperature',
                ascending=[1,0,0,0,1])\
      .show(5)

+----+-----+----+----+-----------+
|  id|annee|mois|jour|temperature|
+----+-----+----+----+-----------+
|7005| 2020|  12|  31|        0.0|
|7005| 2020|  12|  31|        0.2|
|7005| 2020|  12|  31|        1.7|
|7005| 2020|  12|  31|        2.4|
|7005| 2020|  12|  31|        2.4|
+----+-----+----+----+-----------+
only showing top 5 rows



In [42]:
meteo.where('id < 8000')\
     .select('annee','mois_jour',
             'temperature')\
     .describe().show(20)

+-------+------+------------------+------------------+
|summary| annee|         mois_jour|       temperature|
+-------+------+------------------+------------------+
|  count|122248|            122248|            119516|
|   mean|2020.0| 666.2442493946731|13.526039191405314|
| stddev|   0.0|345.16250616182487| 6.969180673312196|
|    min|  2020|              0101|              -7.6|
|    max|  2020|              1231|              39.8|
+-------+------+------------------+------------------+



In [43]:
meteo.count(), meteo.sample(withReplacement=True, fraction=0.01, seed=123456).count()

(174227, 1760)

In [44]:
meteo.sample(True,1/100).count()

1731

In [45]:
meteo.sample(1/100).count()

1740